## Train polyphony encoding

In [39]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [40]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [42]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

We use the music21 corpus for the bach training data. </br>
You can adjust the `paths` to reduce the number of training songs. </br>
</br>
See: https://web.mit.edu/music21/doc/about/referenceCorpus.html

To load custom training data use:
```python
# Load songs
from pathlib import Path

paths = list(Path("./dataset/Pokemon").rglob("*.midi"))
```

In [43]:
from pathlib import Path
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))
# Load songs
# paths = m21.corpus.getComposer('bach')
# paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Found 16 songs in corpus.
Songs encoded: 16


In [44]:
# Create dataset
utils.create_train_data(encoded_songs, "raw_train_ds.txt")
print("Dataset created")

Dataset created


In [45]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)

logdir = "logs/bach_transformer"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train("raw_train_ds.txt", min_training_seq_len=32, epochs=100, tensorboard_callback=tensorboard_callback)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        780288    
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_decoder_4 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_5 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_6 (Tran  (None, None, 256)        3947

In [46]:
%tensorboard --logdir logs/bach_transformer

In [47]:
model.save("Videospielmusik_polyphonie_transformer")

'c:\\Users\\flori\\anaconda3\\envs\\tf\\lib\\site-packages\\mukkeBude\\model\\preTrainedModels\\Videospielmusik_polyphonie_transformer.h5'

## Generate polyphony

In [48]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [49]:
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [50]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

**This does not work! In the current state you can not load a transformer network from a file!**</br>
You need to train it first and use it here.

In [51]:
#model = MukkeBudeTransformer.load(mapping, "Bach_polyphonie_transformer")

In [52]:
# Create song
generated_song = model.generate("n46 d4 xxsep d4 n53 d2 n50 d2 xxsep d2 n62 d1 n58 d1 xxsep d2 n53 d2 n53 d2 n50 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2", max_length=1000, probability=0.8)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))
print(generated_song)

prompt_tokens: [39 19  4 19 32 18 26 18  4 18 20 17 28 17  4 18 32 18 32 18 26 18  4 19
 39 19  4 19 31 18 41 18  4 18 37 17 28 17  4 18]


: 

: 

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song_ints = np.array(new_song_ints)

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=140)

path = Path("generated_song_pokemon_trans_poly.midi")
utils.write_midi(new_song, path)
